In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve
from answer_judge.vqaEval import VQAEval

In [2]:
data_reader=ReadData('/root/autodl-fs/hallscope/VizWiz/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,answer_type,answers,most_likely
0,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,[Given the question 'What is the expiration da...,[unanswerable],"[{'answer_confidence': ['maybe'], 'answer': ['...",{'embedding': [[[-0.00073624 0.0111084 -0.01...
1,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,[Given the question 'For how long do I cook th...,[other],"[{'answer_confidence': ['yes'], 'answer': ['un...",{'embedding': [[[-0.00073624 0.0111084 -0.01...
2,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,"[Given the question 'What is it?', is the ques...",[other],"[{'answer': ['unanswerable'], 'answer_confiden...",{'embedding': [[[-0.00073624 0.0111084 -0.01...
3,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,"[Given the question 'What is this?', is the qu...",[other],"[{'answer': ['clock'], 'answer_confidence': ['...",{'embedding': [[[-0.00567627 -0.00067902 -0.00...
4,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,[Given the question 'What is this? What is thi...,[other],"[{'answer_confidence': ['yes'], 'answer': ['wa...",{'embedding': [[[-0.00073624 0.0111084 -0.01...


In [5]:
df.shape

(4110, 5)

In [6]:
i=1

In [7]:
df.iloc[i]['img_path']

['/root/autodl-fs/vizwiz/train/train/VizWiz_train_00002315.jpg']

In [8]:
df.iloc[i]['most_likely']['response']

['Unanswerable']

In [9]:
df.iloc[i]['question']

["Given the question 'For how long do I cook this in the microwave?', is the question answerable or unanswerable based on the image?\nPlease reply with 'Unanswerable' or 'Answerable'."]

In [10]:
df.iloc[i]['answer_type']

['other']

In [11]:
df['answer_type'].value_counts()

answer_type
[other]           2771
[unanswerable]    1102
[yes/no]           172
[number]            65
Name: count, dtype: int64

In [12]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [13]:
embeddings=np.concatenate(embeddings,axis=0)

In [14]:
embeddings.shape

(4110, 33, 4096)

In [15]:
def transpose_lines(x):
    f=pd.DataFrame(x)
    transposed=f['answer'].apply(pd.Series).T.values.tolist()
    return transposed

df['answers']=df['answers'].apply(transpose_lines)

KeyboardInterrupt: 

In [122]:
gt_label=df['answers'].to_list()
gt_label=np.concatenate(gt_label,axis=0).tolist()

In [123]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [124]:
rsps[0]

'Dial'

In [125]:
judge=VQAEval(gt_label,rsps)
judge.evaluate()
print(judge.evalQA)
print(judge.accuracy['overall'])

computing accuracy


100%|██████████| 1347/1347 [00:00<00:00, 2315.93it/s]

Done computing accuracy
{0: 0.0, 1: 100, 2: 0.0, 3: 0.0, 4: 100, 5: 0.0, 6: 100, 7: 100, 8: 0.0, 9: 0.0, 10: 0.0, 11: 100, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 100, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 100, 23: 100, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 100, 29: 100, 30: 0.0, 31: 100, 32: 66.67, 33: 0.0, 34: 0.0, 35: 100, 36: 33.33, 37: 0.0, 38: 100, 39: 0.0, 40: 0.0, 41: 0.0, 42: 0.0, 43: 100, 44: 0.0, 45: 100, 46: 100, 47: 0.0, 48: 0.0, 49: 100, 50: 100, 51: 0.0, 52: 0.0, 53: 100, 54: 0.0, 55: 33.33, 56: 100, 57: 33.33, 58: 33.33, 59: 100, 60: 100, 61: 100, 62: 33.33, 63: 66.67, 64: 66.67, 65: 0.0, 66: 33.33, 67: 0.0, 68: 100, 69: 33.33, 70: 100, 71: 100, 72: 0.0, 73: 0.0, 74: 0.0, 75: 0.0, 76: 66.67, 77: 66.67, 78: 0.0, 79: 0.0, 80: 100, 81: 100, 82: 100, 83: 0.0, 84: 0.0, 85: 0.0, 86: 0.0, 87: 0.0, 88: 0.0, 89: 0.0, 90: 100, 91: 100, 92: 100, 93: 100, 94: 100, 95: 100, 96: 0.0, 97: 0.0, 98: 0.0, 99: 100, 100: 100, 101: 0.0, 102: 0.0, 103: 0.0, 104: 100, 105: 100, 1

In [126]:
gt_scores=[judge.evalQA[i] for i in range(len(judge.evalQA))]

In [127]:
is_correct=[]
threshold=10
for i,x in enumerate(gt_scores):
    if x >=threshold:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [128]:
is_correct.shape

(1347,)

In [129]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 1347 responses. 726 answer correctly and 621 answer wrongly


In [130]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 1000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:77
validset length:1000
testset length:270
trainset postive: 39 negative: 38
valset postive: 551 negative: 449
testset postive: 136 negative: 134


In [131]:
all_embeddings = embeddings
all_embeddings.shape

(1347, 33, 4096)

In [132]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [133]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [134]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [135]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.6075873386715387


found best n_components:1,layer:7 Score: 0.6086847562035417
found best n_components:1,layer:8 Score: 0.6310332701425632


  9%|▉         | 1/11 [00:02<00:20,  2.08s/it]

found best n_components:2,layer:11 Score: 0.6405644323542132


100%|██████████| 11/11 [00:25<00:00,  2.32s/it]


In [136]:
best_paras

{'n_components': 2, 'layer': 11, 'best_score': np.float64(0.6405644323542132)}

In [137]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [138]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(hall_label_val)

In [139]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.4935064935064935 AUROC:0.5060728744939271


In [140]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.4962962962962963 AUROC:0.5583296751536435


Train Linear Prob to Get Better Results

In [141]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [142]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [143]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.4962962962962963 AUROC:0.46702151009657594
